In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pcs
import cirq
import qudit_gates as qg
import numpy as np

In [9]:
qudits = [cirq.LineQid(i, dimension=3) for i in range(2)]

In [92]:
ancilla_qubit = cirq.LineQubit(2)

error_moment = cirq.Moment(qg.QutritPlusSquaredGate().on(qudits[0]))

noisy_circuit_qubit_ancilla = cirq.Circuit(cirq.H(ancilla_qubit), 
                             qg.QutritZGate().on(qudits[0]).controlled_by(ancilla_qubit),
                             error_moment,
                             qg.QutritZGate().on(qudits[0]).controlled_by(ancilla_qubit),
                             cirq.H(ancilla_qubit), cirq.Moment(cirq.measure(qudits[0]), cirq.measure(ancilla_qubit)))

In [93]:
noisy_circuit_qubit_ancilla

0 (d=3): ───────TZ───[++]───TZ───────M───
                │           │
2: ─────────H───@───────────@────H───M───

In [94]:
pcs.simulate_noisy_circuit(noisy_circuit_qubit_ancilla, shots=100000)

Counter({(1, 1): 100000})

In [114]:
z_unitary = cirq.unitary(qg.ErrorZGate())
y_unitary = cirq.unitary(qg.ErrorYGate())
x_unitary = cirq.unitary(qg.QutritPlusGate())
tz_unitary = cirq.unitary(qg.QutritZGate())
z_squared_unitary = cirq.unitary(qg.ErrorZ2Gate())


In [115]:
z_squared_unitary @ z_unitary

array([[1.+0.00000000e+00j, 0.+0.00000000e+00j, 0.+0.00000000e+00j],
       [0.+0.00000000e+00j, 1.-5.25363176e-16j, 0.+0.00000000e+00j],
       [0.+0.00000000e+00j, 0.+0.00000000e+00j, 1.-9.09955713e-16j]])

In [19]:
x_unitary

array([[0, 0, 1],
       [1, 0, 0],
       [0, 1, 0]])

#TODO: -Why does it still work even when the error does not anticommute? 

- Location of the non-zero matrix entry. What does that "correspond to" ?
- Try creating a matrix whose anticommutator is all zeros but with a non-zero matrix entry within the diagonal. Does it still work?
- Perhaps see if there's some other type of phase error that you are missing.
- Carefully go over literature.
- Set up call with Kate. 

In [14]:
qubits = cirq.LineQubit.range(2) 

# pcs_circuit = cirq.Circuit(cirq.H(qubits[1]), cirq.CX(qubits[1], qubits[0]), cirq.Z(qubits[0]), 
#                            cirq.CX(qubits[1], qubits[0]), cirq.H(qubits[1]), cirq.measure_each(*qubits))

# pcs_circuit

In [15]:
import sympy as sp

def find_anticommuting_matrices(A):
    """
    Finds all matrices X that anticommute with a given matrix A, i.e., AX + XA = 0.

    Parameters:
        A (sympy.Matrix): The given square matrix.

    Returns:
        list[sympy.Matrix]: A list of basis matrices for the solution space.
    """
    # Ensure A is a square matrix
    n = A.shape[0]
    if A.shape[0] != A.shape[1]:
        raise ValueError("Matrix A must be square.")

    # Define a symbolic n x n matrix X
    X = sp.Matrix(sp.MatrixSymbol("x", n, n))

    # Define the anticommutation equation: AX + XA = 0
    equation = A * X + X * A

    # Flatten the equation into a vector of n^2 equations
    equations = equation.reshape(n**2, 1)

    # Extract individual variables from X
    variables = list(X)

    # Solve the linear system
    solutions = sp.linear_eq_to_matrix(equations, variables)
    coeff_matrix, const_vector = solutions
    null_space = coeff_matrix.nullspace()

    # Convert nullspace vectors into basis matrices
    basis_matrices = []
    for vec in null_space:
        basis_matrix = sp.Matrix(n, n, vec)
        basis_matrices.append(basis_matrix)

    return basis_matrices

# Example usage:
A = cirq.unitary(qg.QutritZGate())  # Example 2x2 matrix
anticommuting_matrices = find_anticommuting_matrices(A)
print("Basis for anticommuting matrices:")
for matrix in anticommuting_matrices:
    sp.pprint(matrix)

Basis for anticommuting matrices:
⎡0  1  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦
⎡0  0  1⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣0  0  0⎦
⎡0  0  0⎤
⎢       ⎥
⎢1  0  0⎥
⎢       ⎥
⎣0  0  0⎦
⎡0  0  0⎤
⎢       ⎥
⎢0  0  0⎥
⎢       ⎥
⎣1  0  0⎦


In [113]:
omega**(3)

(0.9999999999999998-4.996003610813204e-16j)

In [110]:
cirq.unitary(qg.ErrorZ2Gate().on(qudits[0]).controlled_by(ancilla_qubit))

array([[ 1. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  1. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  1. +0.j       ,
         0. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         1. +0.j       ,  0. +0.j       ,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       , -0.5-0.8660254j,  0. +0.j       ],
       [ 0. +0.j       ,  0. +0.j       ,  0. +0.j       ,
         0. +0.j       ,  0. +0.j       , -0.5+0.8660254j]])